In [3]:
import os
from time import time
import numpy as np
import scipy
from scipy import linalg
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, cpu_count
# image processing
import cv2
from skimage.measure import label,find_contours,regionprops
from skimage.morphology import remove_small_objects
from sklearn.neighbors import NearestNeighbors
# custom function
from xml2mask import xml2mask
from rgb2hed_v1 import separate_stains
from PIL import Image
from copy import deepcopy

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\kyuha\\PycharmProjects\\TeamKyu\\Sam'

In [ ]:
svs_src = r'\\motherserverdw\Kyu_Sync\Aging\data\svs'
fns = [os.path.splitext(_)[0] for _ in os.listdir(svs_src) if _.lower().endswith('svs')]
dst = os.path.join(svs_src,'roi')

In [ ]:
for fn in fns[0:1]:
    rois = []
    rois = xml2mask(svs_src,fns,dst)


In [ ]:
# for regionidx,region in enumerate(rois):
region = rois #[0]
regionidx = 0

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(region)

In [ ]:
# RGB to Haematoxylin-Eosin-DAB (HED) color space conversion.
# Hematoxylin + Eosin + DAB
start=time()
# rgb_from_hed = np.array([[0.650, 0.704, 0.286],
#                          [0.072, 0.990, 0.105],
#                          [0.268, 0.570, 0.776]])
# hed_from_rgb = scipy.linalg.inv(rgb_from_hed)

rgb_from_hed = np.array([[0.650, 0.704, 0.286],
                         [0.072, 0.990, 0.105],
                         [0.0, 0.0, 0.0]])
rgb_from_hed[2, :] = np.cross(rgb_from_hed[0, :], rgb_from_hed[1, :])
hed_from_rgb = linalg.inv(rgb_from_hed)
Hema = separate_stains(region, hed_from_rgb)[:, :, 0]
print("color deconvolution: {:.2f} sec elapsed".format(time()-start))

In [ ]:
# remove this for production mode #######################
fig, axes = plt.subplots(1, 2, figsize=(24, 12), sharex=True, sharey=True)

a=0*1000
b=4*1000
c=0*1000
d=4*1000

ax = axes.ravel()
ax[0].imshow(region[a:b,c:d])
ax[0].set_title("Original image")

ax[1].imshow(Hema[a:b,c:d], cmap='gray')
ax[1].set_title("ED")

for aa in ax.ravel():
    aa.axis('off')

fig.tight_layout()

In [ ]:
# normalize image between 0 to 1
Hematoxylin = cv2.normalize(Hema[a:b,c:d], None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [ ]:
vectorized = Hematoxylin.reshape((-1,1))
vectorized = np.float32(vectorized)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 8
attempts=10
ret,Klabel,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
center = np.uint8(center)

In [ ]:
Kmean_pixels = []
for i in range(K):
    Klabel2 = deepcopy(Klabel)
    Klabel2[Klabel2!=i]=0
    res = center[Klabel2.flatten()]
    result_image = res.reshape((Hematoxylin.shape))
    Kmean_pixels.append(np.unique(result_image)[-1])





In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(24, 24), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(Hematoxylin[2000:3000,2000:3000],cmap='gray')
ax[0].set_title("Original image")

for idx,aa in enumerate(ax[1:]):
    Klabel2 = deepcopy(Klabel)
    Klabel2[Klabel2!=idx]=0
    res = center[Klabel2.flatten()]
    result_image = res.reshape((Hematoxylin.shape))
    print(np.unique(result_image)[-1])
    aa.imshow(result_image[2000:3000,2000:3000],cmap='gray')
    aa.set_title("K={:d}".format(idx+1))

for aa in ax.ravel():
    aa.axis('off')

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(Hematoxylin, cmap='gray')

In [ ]:
Hematoxylin_temp = deepcopy(Hematoxylin)

In [ ]:
# plt.figure(figsize=(12,12))
# a = Hematoxylin[4000:6000,6000:8000]
# b = a[700:900,1500:1700]
# plt.imshow(b, cmap='gray')
# plt.hist(b)

In [ ]:
#1 intensity thresholding
threshold = 100
Hematoxylin_temp[Hematoxylin<threshold]=0
Hematoxylin_temp[Hematoxylin>threshold]=1
plt.figure(figsize=(12,12))
plt.imshow(Hematoxylin_temp, cmap='gray')

In [ ]:
Hematoxylin = Hematoxylin_temp

In [ ]:
# label the objects
labeled_bw = label(Hematoxylin)
fig,ax=plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(region[a:b,c:d],cmap='gray')
bw = (labeled_bw>0)*1
ax[1].imshow(bw[a:b,c:d],cmap='gray')

In [ ]:
# before size filter
start=time()
prop = regionprops(labeled_bw)
area = [np.sum(x._label_image[x._slice] == x.label) for x in prop]
# narrow down field of view to cells under size 50
area = np.array(area)
area = area[area<50]
# area2 = Parallel(n_jobs=-4, prefer="threads")(delayed(np.sum)(x._label_image[x._slice] == x.label) for x in prop2)
print("{:.2f} sec elapsed".format(time()-start))
plt.hist(area,bins=100)
plt.title('Area',fontsize=30)

In [ ]:
#2 size filter
start=time()
labeled_bw_temp = deepcopy(labeled_bw)
remove_small_objects(labeled_bw_temp,min_size=30,connectivity=1,in_place=True)
print("{:.2f} sec elapsed".format(time()-start))
#plot the result
fig,ax=plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(region[a:b,c:d],cmap='gray')
bw = (labeled_bw_temp>0)*1
ax[1].imshow(bw[a:b,c:d],cmap='gray')

In [ ]:
labeled_bw=labeled_bw_temp
# after size filter
start=time()
prop = regionprops(labeled_bw)
area = [np.sum(x._label_image[x._slice] == x.label) for x in prop]
# area = Parallel(n_jobs=-1, prefer="threads")(delayed(np.sum)(x._label_image[x._slice] == x.label) for x in prop2)
print("{:.2f} sec elapsed".format(time()-start))
plt.hist(area,bins=100)
plt.xlim([0,200])
plt.title('Area',fontsize=30)

In [ ]:
# before AR filter
start=time()
prop = regionprops(labeled_bw)
labels = [_.label for _ in prop]
ARs = [_['major_axis_length']/_['minor_axis_length'] if (_['minor_axis_length']!=0) else 1 for _ in prop]
# def regionar(x):
#     return x['major_axis_length']/x['minor_axis_length']
# ARs = Parallel(n_jobs=-4, prefer="threads")(delayed(regionar)(x) for x in prop)
print("{:.2f} sec elapsed".format(time()-start))
plt.hist(ARs,bins=100)
plt.xlim([0,3])
plt.title('Aspect Ratio',fontsize=30)

In [ ]:
labeled_bw_temp = deepcopy(labeled_bw)

In [ ]:
bw = (labeled_bw_temp>0)*1
plt.figure(figsize=(12,12))
plt.imshow(bw[a:b,c:d],cmap='gray')

In [ ]:
len(np.unique(labeled_bw_temp))

In [ ]:
#3 AR filter
minAR = 2.5
maxAR = 15
start=time()
# def ARfilter(label,AR):
#     if AR<minAR: labeled_bw_temp[labeled_bw==label]=0
#     if AR>maxAR: labeled_bw_temp[labeled_bw==label]=0
# Parallel(n_jobs=-4,prefer="threads")(delayed(ARfilter)(label,AR) for label,AR in zip(labels,ARs))
# for label,AR in zip(labels,ARs):
#     if AR<minAR: labeled_bw_temp[labeled_bw==label]=0
#     if AR>maxAR: labeled_bw_temp[labeled_bw==label]=0
props = regionprops(labeled_bw_temp)
for idx,prop in enumerate(props):
    if idx%100==0: print(idx)
    if (prop['minor_axis_length']!=0): AR = prop['major_axis_length']/prop['minor_axis_length']
    else: AR=0
    if AR<minAR: labeled_bw_temp[labeled_bw_temp==prop.label]=0
    if AR>maxAR: labeled_bw_temp[labeled_bw_temp==prop.label]=0
print("{:.2f} sec elapsed".format(time()-start))

In [ ]:
# after AR filter
prop = regionprops(labeled_bw_temp)
ARs = [_['major_axis_length']/_['minor_axis_length'] for _ in prop]
plt.hist(ARs,bins=100)
plt.title('Aspect Ratio',fontsize=30)

In [ ]:
labeled_bw = labeled_bw_temp

In [ ]:
# before SF filter
SFs = [4*np.pi*_['area']/_['perimeter']**2 for _ in prop]
plt.hist(SFs,bins=100)
plt.title('Circularity',fontsize=30)

In [ ]:
labeled_bw_temp = deepcopy(labeled_bw)

In [ ]:
#4 SF filter (keep cells in the range)
minSF = 0.1
maxSF = 0.7
for prop in regionprops(labeled_bw):
    SF = 4*np.pi*prop['area']/prop['perimeter']**2
    if SF<minSF: labeled_bw_temp[labeled_bw==prop.label]=0;
    if SF>maxSF: labeled_bw_temp[labeled_bw==prop.label]=0;

In [ ]:
# after SF filter
prop = regionprops(labeled_bw_temp)
SFs = [4*np.pi*_['area']/_['perimeter']**2 for _ in prop]
plt.hist(SFs,bins=100)
plt.title('Circularity',fontsize=30)

In [ ]:
labeled_bw = labeled_bw_temp

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(region[a:b,c:d],cmap='gray')
bw = (labeled_bw>0)*1 # *1 to switch dtype from bool to int32
ax[1].imshow(bw[a:b,c:d],cmap='gray')

In [ ]:
bw = (labeled_bw>0)*255
bw_img = Image.fromarray(bw).convert('1')
bw_img.save(os.path.join(dst,fn+'_bw_{:d}.tif'.format(regionidx)))

In [ ]:
labeled_bw_temp = deepcopy(labeled_bw)

In [ ]:
#distance filter
min_dist_to_neighbor = 100
Y = [_.centroid for _ in prop]
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(Y)
distances, indices = nbrs.kneighbors(Y)
close_objects = np.array(prop)[distances[:,1]<min_dist_to_neighbor]
for close_object in close_objects:
    labeled_bw_temp[labeled_bw_temp==close_object.label]=0

In [ ]:
labeled_bw = labeled_bw_temp

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,10))
ax[0].imshow(region[a:b,c:d],cmap='gray')
bw= (labeled_bw>0)*255
ax[1].imshow(bw[a:b,c:d],cmap='gray')

In [ ]:
bw_img = Image.fromarray(bw).convert('1')
bw_img.save(os.path.join(dst,fn+'_bw_far_{:d}.tif'.format(regionidx)))

In [ ]:
# get centroid of each nucleus using regionprop function
# create subfolder to save the crops
props = regionprops(bw)
for idx,prop in enumerate(props):
    centroid = prop['centroid']
    l,r,t,b = (centroid[0]-48,centroid[0]+48,centroid[1]-48,centroid[1]+48)
    croprgb = region[l,r,t,b]
    cropbw = bw[l,r,t,b]
    croprgb_im = Image.fromarray(croprgb)
    cropbw_im = Image.fromarray(cropbw)
    croprgb_im.save('waifko{:d}.tif'.format(idx))
    cropbw_im.save('sdaafvk.tif')


In [ ]:
import pandas as pd
prop = regionprops(labeled_bw)
xs = [np.around(_['centroid'][0]) for _ in prop]
ys = [np.around(_['centroid'][1]) for _ in prop]
area = [np.sum(x._label_image[x._slice] == x.label) for x in prop]
ARs = [_['major_axis_length']/_['minor_axis_length'] for _ in prop]
SFs = [4*np.pi*_['area']/_['perimeter']**2 for _ in prop]
dict = {'x':xs,'y':ys,'area':area,'aspect_ratio':ARs,'circularity':SFs}
df = pd.DataFrame(dict)
df.to_csv(os.path.join(dst,fn+'_parameters_{:d}.csv'.format(regionidx)))